In [1]:
!pip install -r requirements.txt

In [2]:
import uuid

# Generate a unique run ID for these experiments
run_uid = uuid.uuid4().hex[:6]

In [3]:
import os

os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# Silence warnings from HuggingFace
os.environ["TOKENIZERS_PARALLELISM"] = "false"

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"]="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"
os.environ["LANGCHAIN_PROJECT"]="pr-charming-platinum-50"

In [4]:
from langchain_benchmarks import clone_public_dataset, registry

registry.filter(Type="RetrievalTask")

/home/jadson/Documentos/ia2/.conda/lib/python3.11/site-packages/langchain_benchmarks/extraction/tasks/chat_extraction/evaluators.py:8: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_benchmarks.extraction.tasks.chat_extraction.schema import GenerateTicket
/home/jadson/Documentos/ia2/.conda/lib/python3.11/site-packages/langchain_benchmarks/tool_usage/agents/__init__.py:11: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydant

Name,Type,Dataset ID,Description
LangChain Docs Q&A,RetrievalTask,452ccafc-18e1-4314-885b-edd735f17b9d,Questions and answers based on a snapshot of the LangChain python docs. The environment provides the documents and the retriever information. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Semi-structured Reports,RetrievalTask,c47d9617-ab99-4d6e-a6e6-92b8daf85a7d,Questions and answers based on PDFs containing tables and charts. The task provides the raw documents as well as factory methods to easily index them and create a retriever. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Multi-modal slide decks,RetrievalTask,40afc8e7-9d7e-44ed-8971-2cae1eb59731,This public dataset is a work-in-progress and will be extended over time. Questions and answers based on slide decks containing visual tables and charts. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer.


In [5]:
langchain_docs = registry["LangChain Docs Q&A"]
langchain_docs

Name,LangChain Docs Q&A
Type,RetrievalTask
Dataset ID,452ccafc-18e1-4314-885b-edd735f17b9d
Description,Questions and answers based on a snapshot of the LangChain python docs. The environment provides the documents and the retriever information. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Retriever Factories,"basic, parent-doc, hyde"
Architecture Factories,conversational-retrieval-qa
get_docs,


In [6]:
clone_public_dataset(langchain_docs.dataset_id, dataset_name=langchain_docs.name)

  0%|          | 0/91 [00:00<?, ?it/s]

Finished fetching examples. Creating dataset...
New dataset created you can access it at https://smith.langchain.com/o/18c58030-ad8e-47f2-af54-39d4e63eb797/datasets/f858be38-1c7c-49ec-979e-08b7801265ca.
Done creating dataset.


In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-base",
    model_kwargs={"device": 0},  # Comment out to use CPU
)

retriever_factory = langchain_docs.retriever_factories["basic"]
# Indexes the documents with the specified embeddings
# Note that this does not apply any chunking to the docs,
# which means the documents can be of arbitrary length
retriever = retriever_factory(embeddings)

0it [00:00, ?it/s]

In [17]:
# Factory for creating a conversational retrieval QA chain

chain_factory = langchain_docs.architecture_factories["conversational-retrieval-qa"]

In [18]:
from functools import partial

from langsmith.client import Client

from langchain_benchmarks.rag import get_eval_config

In [20]:
from langchain.chat_models import ChatOllama

# A llama2-based model with 128k context
# (in theory) In practice, we will see how well
# it actually leverages that context.
ollama = ChatOllama(model="llama3.2")

/tmp/ipykernel_10909/1057927251.py:6: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  ollama = ChatOllama(model="llama3.2")


In [ ]:
client = Client(llm=ollama)

RAG_EVALUATION = get_eval_config()

TypeError: Client.__init__() got an unexpected keyword argument 'llm'

In [ ]:
chain_factory(retriever, llm=ollama).invoke({"question": "what's lcel?"})

In [ ]:
ollama.list_models()

In [ ]:
# We'll go back to the GTE embeddings for now

retriever_factory = langchain_docs.retriever_factories["basic"]
retriever = retriever_factory(embeddings)

In [ ]:
ollama_test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=partial(chain_factory, llm=ollama, retriever=retriever),
    evaluation=RAG_EVALUATION,
    project_name=f"llama3.2 qa-chain basic {run_uid}",
    project_metadata={
        "index_method": "basic",
        "embedding_model": "thenlper/gte-base",
        "llm": "ollama/llama3.2",
    },
    verbose=True,
)

In [ ]:
ollama_test_run.get_aggregate_feedback()